<h1 style="text-align: center;"><a title="Data Science-AIMS-Cmr-2021-22">Neural Networks for Classification</h1>

## <font color="green"> Learning outcomes:

* Implementing a neural network for feature based classification

* Using the ```CategoricalCrossentropy``` loss

* How to use ```EarlyStopping``` to prevent overfitting.

* Uses the build in softmax function

* Obtaining confusion matrix

## <font color="green">Data information:

* Features: 4 real-valued features

* Output: 3-class target represented by strings

## <font color="green">Tasks for participants (boolean)?

* No, follow along.

## Various Python imports

In [ ]:
import numpy as np
np.random.seed(1337)
import pandas

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import metrics
from keras.utils import np_utils
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import plot_model
from keras.callbacks import EarlyStopping


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.datasets import load_iris

import matplotlib.pyplot as plt
%matplotlib inline

## Load the dataset

This data sets consists of 3 different types of irises’ (Setosa, Versicolour, and Virginica) petal and sepal length, stored in a 150x4 numpy.ndarray

The rows being the samples and the columns being: Sepal Length, Sepal Width, Petal Length and Petal Width.

The dataset is obtained from scikit learn https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html

In [ ]:
iris_data = load_iris()

The load_iris() returns a 'sklearn.utils.Bunch' which is kind of like a Python dictionary with the following keys. In our case we don't need all of that information, we really just need the actual data (i.e. features and class variables). 

Bunch objects are sometimes used as an output for functions and methods. They extend dictionaries by enabling values to be accessed by key, bunch["value_key"], or by an attribute, bunch.value_key.

If you are interested in the Bunch object, here is the API https://scikit-learn.org/stable/modules/generated/sklearn.utils.Bunch.html

In [ ]:
type(iris_data)

In [ ]:
iris_data.keys()

Nonetheless let's take a look at everything that we get when we call the iris_data() function.

In [ ]:
iris_data

Now let's extract what we really want, the data, which is stored in the 'data' key in thie iris_data dictionary. Calling the data key will provide us with the features in this case. Remember, this dataset is formatted in this particular way. Not every dataset will be stored in this way.

In [ ]:
dataset = iris_data.data

## View the data

In [ ]:
dataset

## Scaling

You might already know that neural networks perform best when the features are scaled/normalised instead of being on different scales. Let's take a look and check if the features are on similar scales.

In [ ]:
plt.hist(dataset[:,0])
plt.title('Feature 0')
plt.show()

In [ ]:
print ('mean feature 0:',np.mean(dataset[:,0]))
print ('sdev feature 0:',np.std(dataset[:,0]))

In [ ]:
plt.hist(dataset[:,1])
plt.title('Feature 1')
plt.show()

In [ ]:
print ('mean feature 1:',np.mean(dataset[:,1]))
print ('sdev feature 1:',np.std(dataset[:,1]))

In [ ]:
plt.hist(dataset[:,2])
plt.title('Feature 2')
plt.show()

In [ ]:
print ('mean feature 2:',np.mean(dataset[:,2]))
print ('sdev feature 2:',np.std(dataset[:,2]))

In [ ]:
plt.hist(dataset[:,3])
plt.title('Feature 3')
plt.show()

In [ ]:
print ('mean feature 3:',np.mean(dataset[:,3]))
print ('sdev feature 3:',np.std(dataset[:,3]))

## Now let's scale the data

In [ ]:
scaler = StandardScaler()
scaler.fit(dataset)
scaled_dataset = scaler.transform(dataset)

In [ ]:
plt.hist(scaled_dataset[:,0])
plt.title('Feature 0')
plt.show()

In [ ]:
print ('mean feature 0:',np.mean(scaled_dataset[:,0]))
print ('sdev feature 0:',np.std(scaled_dataset[:,0]))

In [ ]:
plt.hist(scaled_dataset[:,1])
plt.title('Feature 1')
plt.show()

In [ ]:
print ('mean feature 1:',np.mean(scaled_dataset[:,1]))
print ('sdev feature 1:',np.std(scaled_dataset[:,1]))

In [ ]:
plt.hist(scaled_dataset[:,2])
plt.title('Feature 2')
plt.show()

In [ ]:
print ('mean feature 2:',np.mean(scaled_dataset[:,2]))
print ('sdev feature 2:',np.std(scaled_dataset[:,2]))

In [ ]:
plt.hist(scaled_dataset[:,3])
plt.title('Feature 3')
plt.show()

In [ ]:
print ('mean feature 3:',np.mean(scaled_dataset[:,3]))
print ('sdev feature 3:',np.std(scaled_dataset[:,3]))

## Check the shape of the data

It is generally a good idea to check your data once you conduct some manipulation.

In [ ]:
scaled_dataset.shape

Okay so we have 150 examples and 4 features. Next we need to create our X and Y variables.

## Split the data into features and the class values

The dataset above (when we loaded it) had another key in the 'Bunch' called 'target'. So we can call that key and get the values of the classes.

In [ ]:
X = scaled_dataset
Y = iris_data.target

Let's take a look at the unique values in the Y target variable

In [ ]:
np.unique(Y)

Okay so we cannot leave the targets as integer values. We don't want a neural network to assume that 2 is greater than 1 or that the targets have ordinal meaning (1 > 0, 2> 1 etc...). 

We also cannot train a neural network on categorical data represented as text. For example in this example, [0,1,2] corresponds to ['setosa', 'versicolor', 'virginica']. If instead of the numbers, the Y variable had strings of classes (e.g. Y = ['setosa', 'setosa', 'versicolor' ...  , 'versicolor'])

To deal with this, we can convert out targets into one-hot encoded vectors. Keras has a function called 'np_utils.to_categorical()' which helps us achieve this.

## One hot encoding

In [ ]:
Y = np_utils.to_categorical(Y)

Let's take a look at the first 5 examples after applying the encoding:

In [ ]:
Y[0:5]

Okay so clearly the targets have been converted into their one-hot encoded vectors! Great, now let's double check the shapes of X and Y before creatining our training, validation and test splits.

## Check the shapes of the data

In [ ]:
X.shape

In [ ]:
Y.shape

## Split the data into training and test data

After inspection, the shapes look correct and the target values look correct too (dimensions of 3 since we have 3 unique classes). Once we have our data into X (features) and Y (class) variables we can proceed to the next step which is to create training and testing data. This is similar to the previous notebook.

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.3)

## Create a neural network model

Some points to keep in mind everytime you define a model:

* Remember to specify and intput dimension for the first layer (this is always  the case for the first layer only)
* You will have to define a model architecture, how many layers, how many units etc...
* You will have to compile the model, specify a loss function and optimiser. The metric is not used in any calculation, it is an easy-to-use evaluation metric for us humans to monitor the model. API documentation on metrics is here: https://www.tensorflow.org/api_docs/python/tf/keras/metrics

We specify the loss function as a Tensorflow.keras.losses object, in this case categorical cross entropy loss. The API documentation is here: https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy 

In the examples so far we have been calling activation functions by their pre-defined names such as 'linear', 'relu' and 'softmax'. But other activation functions exist. The formal document is available, here is an example for ReLU https://www.tensorflow.org/api_docs/python/tf/keras/activations/relu and you can see that we can use it as: tf.keras.activations.relu we can import it and then use it as follows:

```from tensorflow.keras.activations import relu```

and then we would use it within a layer as follows

```model.add(Dense(2, input_dim=4, activation=relu, kernel_initializer = initializer))```

Notice in the above example we wrote ```activation=relu``` instead of ```activation='relu'```. 


In [ ]:
def baseline():
  
    # Seeting a seed so we can reproduce the results
    tf.random.set_seed(137)

    # Define the initialisation method
    initializer = RandomNormal(mean = 0.0, stddev=0.01)

    # create a squential model
    model = Sequential()

    # add one fully connected layer
    model.add(Dense(2, input_dim=4, activation='relu', kernel_initializer = initializer))

    # add one fully connected output layer with a softmax activation
    model.add(Dense(3, activation='softmax', kernel_initializer = initializer))

    # Define the optimiser
    optimiser = Adam(learning_rate=0.001)

    # Define the loss function
    loss = CategoricalCrossentropy()
    
    # Compile model
    model.compile(loss=loss, 
                  optimizer=optimiser, 
                  metrics=['accuracy'])
    
    return model

## Initialise the model

To initialise the model, we call our Python function which contains the model.

In [ ]:
model = baseline()

## Determine the number of trainable parameters

In [ ]:
model.summary()

We can also view the model by plotting it. It gives a different perspective if you prefer this view. To do this we use the plot_model() function.

API documentation for the plot_model() function https://www.tensorflow.org/api_docs/python/tf/keras/utils/plot_model?hl=en

In [ ]:
plot_model(model, show_shapes=True)

Let's add something new.

Sometimes when we train models we can end up in situations where the network overfits and only gets worse. One way to handle this is to stop training once the validation loss does not improve over a certain number of epochs. Tensorflow has something called 'EarlyStopping' which does exactly that. Namely, it will stop training when a monitored metric has stopped improving. API: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping?hl=en

In this example, we can monitor the validation accuracy. We want the validation loss to increase so we tell it that the 'mode' is to keep an eye if the validation accuracy is increasing. Formally, "in min mode, training will stop when the quantity monitored has stopped decreasing" The converse for max.

We also specify a patience arguement. Formally, "number of epochs with no improvement after which training will be stopped". In our example, we will wait for 10 epochs.

Here is the code to create this and then we need to add it to our fit function.

In [ ]:
stopping = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=10)

## Fit the model

Now that we have the training, validation and testing features and targets we can start training the network. To do this, we make use of the .fit() function like in the previous notebook. 

Here we add our callback which we created above 'stopping'. A callback is formally defined as "utilities called at certain points during model training." We specify them in an array as we can add multiple callsbacks. In this example we only have one. In this example I specify a high number of epochs, let's see what happens, will it run for 400 epochs or stop before that?

In [ ]:
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs = 400, callbacks=[stopping])

## Plot the loss over the epochs

Keep an eye out for over fitting.

In [ ]:
plt.figure(figsize=(8, 8))
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='validation loss', linestyle='dashed')
plt.title('Model loss')
plt.ylabel('Cross entropy loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy', linestyle='dashed')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

## Predict on the test data

In [ ]:
prediction = model.predict(X_test)

## View predictions

The predictions in this example are different to the previous notebooks. In this example the output is softmax outputs. Note the performance is pretty bad. The model provided above is not necessarily the best model :) Here we display the predictions on the first 5 testing examples

In [ ]:
prediction[0:5]

## Predict classes (instead of softmax output)

Here we use np.argmax() on the predictions to get the corresponding class (i.e. not the softmax but the iteger representations)


In [ ]:
prediction_classes = np.argmax(model.predict(X_test), axis=-1)

Let's take a look at the predictions for the first 10 testing points and compare them to the correct values.

In [ ]:
prediction_classes[0:10]

Y_test is currently in one-hot encoded vector form. We can use the np.argmax() function to convert the vectors back into integer values.

In [ ]:
np.argmax(Y_test,axis=1)[0:10]

## Compute the confusion matrix

Scikit learn has a class called confusion_matrix API: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html?highlight=confusion#sklearn.metrics.confusion_matrix which can compute the confusion matrix for us. This won't work on softmax output so we need to apply ```np.argmax()```. We have already done this for the model predictions. We imported this at the very top of the script.

In [ ]:
confusion_matrix(np.argmax(Y_test,1), prediction_classes)

## Compute the accuracy

Scikit learn has a class called accuracy_score, API: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score which can compute multiclass/binary class accuracy. We imported this at the top of the script. Again, it does not work on softmax values.

Accuracy is suitable in this example but for some examples one might want to consider different performance metrics. Additional options are available, take a look here: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics

In [ ]:
accuracy_score(np.argmax(Y_test,1), prediction_classes)

# Task

Try to improve the model accuracy by modifying the network. 

* You could try adding more layers, using different activation functions or 

* changing the number of units in each fully connected layer. 

The particular network I provided here was deliberately not optimal and thus the performance can definitely be improved :)

# References:

* This notebook was adpated from Dr. Emmanuel Dufourq,  2021 Gene Golub SIAM Summer School 